## Data Transformation


In [2]:
import os
%pwd

'c:\\Users\\tizia\\Documents\\Desktop\\MLOps\\TextSummariser\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\tizia\\Documents\\Desktop\\MLOps\\TextSummariser'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [5]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_path = CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
           root_dir = config.root_dir,
           data_path = config.data_path,
           tokenizer_name = config.tokenizer_name)
        
        return data_transformation_config

In [8]:
import os
from src.TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py
🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py
🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py


### Data Transformation Component

In [9]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config 
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        dataset_samsun = load_from_disk(self.config.data_path)
        dataset_samsun_pt = dataset_samsun.map(self.convert_examples_to_features, batched = True)
        dataset_samsun_pt.save_to_disk(os.path.join(self.config.root_dir, "samsun_dataset"))
                   

In [12]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config=data_transformation_config)
data_transformation.convert()


[2026-01-05 23:06:42,414: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-05 23:06:42,414: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-05 23:06:42,414: INFO: common: created directory at: artifacts]
[2026-01-05 23:06:42,426: INFO: common: created directory at: artifacts/data_transformation]


loading configuration file config.json from cache at C:\Users\tizia\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail\snapshots\40d588fdab0cc077b80d950b300bf66ad3c75b92\config.json
Model config PegasusConfig {
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "forced_eos_token_id": 1,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
  